In [ ]:
# Third-party
import astropy.coordinates as coord
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
c1 = coord.SkyCoord(ra=100*u.deg,
                    dec=-20*u.deg,
                    distance=10*u.kpc,
                    pm_ra_cosdec=10*u.mas/u.yr,
                    pm_dec=10*u.mas/u.yr)

c2 = coord.SkyCoord(ra=100*u.deg,
                    dec=-20*u.deg,
                    distance=10*u.kpc,
                    radial_velocity=150*u.km/u.s)

c3 = coord.SkyCoord(ra=100*u.deg,
                    dec=-20*u.deg,
                    distance=10*u.kpc,
                    pm_ra_cosdec=10*u.mas/u.yr,
                    pm_dec=10*u.mas/u.yr,
                    radial_velocity=150*u.km/u.s)

coord.concatenate((c1, c1))

In [ ]:
c2.get_representation_component_names('s')

In [ ]:
c2.data.differentials['s']

In [ ]:
c2.data.

In [ ]:
coord.SkyCoord((c1, c1))

In [ ]:
icrs = coord.ICRS()

In [ ]:
has_diff = 's' in c1.data.differentials

In [ ]:
(icrs.get_representation_component_names(which='base'),
 icrs.get_representation_component_names(which='s'))

In [ ]:
def _parse_coordinate_arg(coords, frame, units, init_kwargs):
    """
    Single unnamed arg supplied.  This must be:
    - Coordinate frame with data
    - Representation
    - SkyCoord
    - List or tuple of:
      - String which splits into two values
      - Iterable with two values
      - SkyCoord, frame, or representation objects.

    Returns a dict mapping coordinate attribute names to values (or lists of
    values)
    """
    is_scalar = False  # Differentiate between scalar and list input
    valid_kwargs = {}  # Returned dict of lon, lat, and distance (optional)

    frame_attr_names = list(frame.representation_component_names.keys())
    repr_attr_names = list(frame.representation_component_names.values())
    repr_attr_classes = list(frame.representation.attr_classes.values())
    n_attr_names = len(repr_attr_names)

    # Turn a single string into a list of strings for convenience
    if isinstance(coords, str):
        is_scalar = True
        coords = [coords]

    if isinstance(coords, (SkyCoord, BaseCoordinateFrame)):
        # Note that during parsing of `frame` it is checked that any coordinate
        # args have the same frame as explicitly supplied, so don't worry here.

        if not coords.has_data:
            raise ValueError('Cannot initialize from a frame without coordinate data')

        data = coords.data.represent_as(frame.representation_type)

        values = []  # List of values corresponding to representation attrs
        repr_attr_name_to_drop = []
        for repr_attr_name in repr_attr_names:
            # If coords did not have an explicit distance then don't include in initializers.
            if (isinstance(coords.data, UnitSphericalRepresentation) and
                    repr_attr_name == 'distance'):
                repr_attr_name_to_drop.append(repr_attr_name)
                continue

            # Get the value from `data` in the eventual representation
            values.append(getattr(data, repr_attr_name))

        # drop the ones that were skipped because they were distances
        for nametodrop in repr_attr_name_to_drop:
            nameidx = repr_attr_names.index(nametodrop)
            del repr_attr_names[nameidx]
            del units[nameidx]
            del frame_attr_names[nameidx]
            del repr_attr_classes[nameidx]

        if coords.data.differentials and 's' in coords.data.differentials:
            orig_vel = coords.data.differentials['s']
            vel = coords.data.represent_as(frame.representation, frame.get_representation_cls('s')).differentials['s']
            for frname, reprname in frame.get_representation_component_names('s').items():
                if (reprname == 'd_distance' and not hasattr(orig_vel, reprname) and
                    'unit' in orig_vel.get_name()):
                    continue
                values.append(getattr(vel, reprname))
                units.append(None)
                frame_attr_names.append(frname)
                repr_attr_names.append(reprname)
                repr_attr_classes.append(vel.attr_classes[reprname])

        for attr in frame_transform_graph.frame_attributes:
            value = getattr(coords, attr, None)
            use_value = (isinstance(coords, SkyCoord)
                         or attr not in coords._attr_names_with_defaults)
            if use_value and value is not None:
                valid_kwargs[attr] = value

    elif isinstance(coords, BaseRepresentation):
        if coords.differentials and 's' in coords.differentials:
            diffs = frame.get_representation_cls('s')
            data = coords.represent_as(frame.representation_type, diffs)
            values = [getattr(data, repr_attr_name) for repr_attr_name in repr_attr_names]
            for frname, reprname in frame.get_representation_component_names('s').items():
                values.append(getattr(data.differentials['s'], reprname))
                units.append(None)
                frame_attr_names.append(frname)
                repr_attr_names.append(reprname)
                repr_attr_classes.append(data.differentials['s'].attr_classes[reprname])

        else:
            data = coords.represent_as(frame.representation)
            values = [getattr(data, repr_attr_name) for repr_attr_name in repr_attr_names]

    elif (isinstance(coords, np.ndarray) and coords.dtype.kind in 'if'
          and coords.ndim == 2 and coords.shape[1] <= 3):
        # 2-d array of coordinate values.  Handle specially for efficiency.
        values = coords.transpose()  # Iterates over repr attrs

    elif isinstance(coords, (Sequence, np.ndarray)):
        # Handles list-like input.

        vals = []
        is_ra_dec_representation = ('ra' in frame.representation_component_names and
                                    'dec' in frame.representation_component_names)
        coord_types = (SkyCoord, BaseCoordinateFrame, BaseRepresentation)
        if any(isinstance(coord, coord_types) for coord in coords):
            # this parsing path is used when there are coordinate-like objects
            # in the list - instead of creating lists of values, we create
            # SkyCoords from the list elements and then combine them.
            scs = [SkyCoord(coord, **init_kwargs) for coord in coords]

            # Check that all frames are equivalent
            for sc in scs[1:]:
                if not sc.is_equivalent_frame(scs[0]):
                    raise ValueError("List of inputs don't have equivalent "
                                     "frames: {0} != {1}".format(sc, scs[0]))

            # Now use the first to determine if they are all UnitSpherical
            allunitsphrepr = isinstance(scs[0].data, UnitSphericalRepresentation)
            
            # Use the first to determine if they have differentials, and the type 
            # of differential
            has_diff = 's' in scs[0].data.differentials
            if has_diff:
                diff_type = type(scs[0].data.differentials['s'])
                print(type(scs[1].data.differentials['s']), diff_type)
                all_same_diffs = all('s' in scs[i].data.differentials and 
                                     type(scs[i].data.differentials['s']) == diff_type
                                     for i in range(1, len(scs)))
                
                print(all_same_diffs)

            # get the frame attributes from the first coord in the list, because
            # from the above we know it matches all the others.  First copy over
            # the attributes that are in the frame itself, then copy over any
            # extras in the SkyCoord
            for fattrnm in scs[0].frame.frame_attributes:
                valid_kwargs[fattrnm] = getattr(scs[0].frame, fattrnm)
            for fattrnm in scs[0]._extra_frameattr_names:
                valid_kwargs[fattrnm] = getattr(scs[0], fattrnm)

            # Now combine the values, to be used below
            values = []
            for data_attr_name, repr_attr_name in zip(frame_attr_names, repr_attr_names):
                if allunitsphrepr and repr_attr_name == 'distance':
                    # if they are *all* UnitSpherical, don't give a distance
                    continue
                data_vals = []
                for sc in scs:
                    data_val = getattr(sc, data_attr_name)
                    data_vals.append(data_val.reshape(1,) if sc.isscalar else data_val)
                concat_vals = np.concatenate(data_vals)
                # Hack because np.concatenate doesn't fully work with Quantity
                if isinstance(concat_vals, u.Quantity):
                    concat_vals._unit = data_val.unit
                values.append(concat_vals)
        else:
            # none of the elements are "frame-like"
            # turn into a list of lists like [[v1_0, v2_0, v3_0], ... [v1_N, v2_N, v3_N]]
            for coord in coords:
                if isinstance(coord, str):
                    coord1 = coord.split()
                    if len(coord1) == 6:
                        coord = (' '.join(coord1[:3]), ' '.join(coord1[3:]))
                    elif is_ra_dec_representation:
                        coord = _parse_ra_dec(coord)
                    else:
                        coord = coord1
                vals.append(coord)  # Assumes coord is a sequence at this point

            # Do some basic validation of the list elements: all have a length and all
            # lengths the same
            try:
                n_coords = sorted(set(len(x) for x in vals))
            except Exception:
                raise ValueError('One or more elements of input sequence does not have a length')

            if len(n_coords) > 1:
                raise ValueError('Input coordinate values must have same number of elements, found {0}'
                                 .format(n_coords))
            n_coords = n_coords[0]

            # Must have no more coord inputs than representation attributes
            if n_coords > n_attr_names:
                raise ValueError('Input coordinates have {0} values but '
                                 'representation {1} only accepts {2}'
                                 .format(n_coords,
                                         frame.representation_type.get_name(),
                                         n_attr_names))

            # Now transpose vals to get [(v1_0 .. v1_N), (v2_0 .. v2_N), (v3_0 .. v3_N)]
            # (ok since we know it is exactly rectangular).  (Note: can't just use zip(*values)
            # because Longitude et al distinguishes list from tuple so [a1, a2, ..] is needed
            # while (a1, a2, ..) doesn't work.
            values = [list(x) for x in zip(*vals)]

            if is_scalar:
                values = [x[0] for x in values]
    else:
        raise ValueError('Cannot parse coordinates from first argument')

    # Finally we have a list of values from which to create the keyword args
    # for the frame initialization.  Validate by running through the appropriate
    # class initializer and supply units (which might be None).
    try:
        for frame_attr_name, repr_attr_class, value, unit in zip(
                frame_attr_names, repr_attr_classes, values, units):
            valid_kwargs[frame_attr_name] = repr_attr_class(value, unit=unit,
                                                            copy=False)
    except Exception as err:
        raise ValueError('Cannot parse first argument data "{0}" for attribute '
                         '{1}'.format(value, frame_attr_name), err)
    return valid_kwargs

In [ ]:
from astropy.coordinates.sky_coordinate import *
from astropy.coordinates.baseframe import BaseCoordinateFrame
from astropy.coordinates.representation import BaseRepresentation, UnitSphericalRepresentation
from collections.abc import Sequence

In [ ]:
# these should all succeed
_parse_coordinate_arg((c1, c1), icrs, [], dict())
_parse_coordinate_arg((c2, c2), icrs, [], dict())
_parse_coordinate_arg((c3, c3), icrs, [], dict())

In [ ]:
# these should all fail
_parse_coordinate_arg((c1, c2), icrs, [], dict())
_parse_coordinate_arg((c2, c3), icrs, [], dict())
_parse_coordinate_arg((c1, c3), icrs, [], dict())

In [ ]:
type(c1.data.differentials['s'])

In [ ]:
type(c2.data.differentials['s'])

In [ ]:
type(c3.data.differentials['s'])

In [ ]:
concatenate((c1.data, c1.data))
concatenate((c2.data, c2.data))
concatenate((c3.data, c3.data))

with pytest.raises(ValueError):
    concatenate((c1.data, c2.data)) # should fail

with pytest.raises(ValueError):
    concatenate((c1.data, c3.data)) # should fail
    
with pytest.raises(ValueError):
    concatenate((c2.data, c3.data)) # should fail

In [ ]:
c1._